# Testing Deployment

For GPU available devices

`%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118`

Otherwise

`%pip uninstall -y torch torchvision torchaudio`

In [2]:
from dotenv import load_dotenv

load_dotenv()

import sys
import os

sys.path.append(os.getenv('SRC_DIR'))

from utils.system import display_system_info

display_system_info(markdown=True)


**Last Updated**: 2025-04-17 00:45:57

**Python Version**: 3.11.11  
**OS**: Windows 10.0.26100  
**Architecture**: 64bit  
**Hostname**: DESKTOP-42J9AQP  
**Processor**: Intel64 Family 6 Model 183 Stepping 1, GenuineIntel  
**RAM Size**: 63.85 GB  
  
        

In [3]:
from ultralytics import YOLO
import cv2
import numpy as np
import pygame

from collections import deque

pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Using Programmatic Input

In [3]:
SIREN_PATH = os.path.join(os.getenv('DATA_DIR'), 'audio', 'siren.mp3')
PERSON_CLASS_ID = 0
DROWNING_CLASS_ID = 2
NUM_CLASSES = 3
HISTORY_LENGTH = 30
DROWNING_THRESHOLD = 15
SENSITIVITY = 0.2  # Sensitivity factor for smoothing, can be dynamically adjusted

def create_class_vector(confidence, class_id, num_classes=80):
    class_vector = np.zeros(num_classes)
    class_vector[class_id] = confidence
    return class_vector

def apply_temporal_smoothing(existing_vector, new_vector, sensitivity=SENSITIVITY):
    if existing_vector is None:
        # If there's no previous value, simply return the new vector
        return new_vector
    else:
        # Apply Exponential Moving Average (EMA) to stabilize
        return sensitivity * existing_vector + (1 - sensitivity) * new_vector

def get_stabilized_object_label(existing_label_vec, confidence, cls, sensitivity=SENSITIVITY, num_classes=80):
    new_label_vec = create_class_vector(confidence, cls, num_classes)
    
    # Apply temporal smoothing to stabilize the label
    return apply_temporal_smoothing(existing_label_vec, new_label_vec, sensitivity)

def get_class_from_label(label_vec):
    return int(np.argmax(label_vec))

def get_stabilized_class_predictions(existing_label_vec, class_confs, sensitivity=SENSITIVITY):
    # Convert class confidence list to a numpy array
    new_label_vec = np.array(class_confs)
    
    # Apply temporal smoothing to stabilize the class predictions
    return apply_temporal_smoothing(existing_label_vec, new_label_vec, sensitivity)

def is_activate_drowning_alert(tracked_info):
    for id, info in tracked_info.items():
        if len(info['frame_history']) < HISTORY_LENGTH:
            continue

        most_common = max(set(info['frame_history']), key=info['frame_history'].count)
        is_drowning = (
            most_common == DROWNING_CLASS_ID and 
            info['frame_history'].count(DROWNING_CLASS_ID) >= DROWNING_THRESHOLD and 
            get_class_from_label(info['obj_label_vec']) == PERSON_CLASS_ID
        )

        if is_drowning:
            return True
        
    return False

def simulate_action_class_prediction():
    randint = np.random.randint(DROWNING_CLASS_ID, NUM_CLASSES)
    return [1 if i == randint else 0 for i in range(NUM_CLASSES)]

def play_siren():
    if not pygame.mixer.music.get_busy():
        pygame.mixer.music.play()

def stop_siren():
    if pygame.mixer.music.get_busy():
        pygame.mixer.music.stop()

In [8]:
pygame.mixer.init()
pygame.mixer.music.load(SIREN_PATH)

yolo_model = YOLO(os.path.join(os.getenv('YOLO_DIR'), 'yolo11n.pt'))

filename = os.path.join(os.getenv('DATA_DIR'), 'videos', 'indoorpool.mp4')
cap = cv2.VideoCapture(filename)

tracked_info = {}

while cap.isOpened():
    drowning_alert = False
    ret, frame = cap.read()

    if not ret:
        print("End of video or cannot read the frame.")
        break

    result = yolo_model.track(frame, persist=True, tracker="botsort.yaml", verbose=False)[0]

    if result.boxes and result.boxes.id is not None:
        boxes = result.boxes.xyxy.cpu().numpy()
        obj_clses = result.boxes.cls.cpu().numpy()
        obj_confs = result.boxes.conf.cpu().numpy()
        ids = result.boxes.id.cpu().numpy()

        for box, cls, conf, id in zip(boxes, obj_clses, obj_confs, ids):
            if id not in tracked_info:
                existing_obj_label_vec = None
                existing_class_prediction_vec = None
                frame_history = deque(maxlen=HISTORY_LENGTH)
            else:
                existing_obj_label_vec = tracked_info[id]['obj_label_vec']
                existing_class_prediction_vec = tracked_info[id]['class_prediction_vec']
                frame_history = tracked_info[id]['frame_history']

            obj_label_vec = get_stabilized_object_label(existing_obj_label_vec, conf, int(cls))
            obj_label = get_class_from_label(obj_label_vec)

            if obj_label == PERSON_CLASS_ID:
                class_confs = simulate_action_class_prediction() # Simulate class prediction for demonstration purposes
                class_prediction_vec = get_stabilized_class_predictions(existing_class_prediction_vec, class_confs, SENSITIVITY)
                class_prediction = get_class_from_label(class_prediction_vec)

                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'Person {id}, Predicted Class: {class_prediction}, Conf: {class_prediction_vec[class_prediction]}, obj_conf: {obj_label_vec[obj_label]}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
                frame_history.append(class_prediction)
            else:
                # print(f"Skipping object with ID {id} and label {obj_label} due to low confidence or not person, obj_conf{obj_label_vec[obj_label]}.")
                class_prediction_vec = existing_class_prediction_vec
                class_prediction = get_class_from_label(class_prediction_vec)

            tracked_info[id] = {
                'obj_label_vec': obj_label_vec,
                'class_prediction_vec': class_prediction_vec,
                'frame_history': frame_history
            }

        drowning_alert = is_activate_drowning_alert(tracked_info)

        if drowning_alert:
            print("Drowning alert!")
            play_siren()
        
    fps = cap.get(cv2.CAP_PROP_FPS)
    cv2.putText(frame, f'FPS: {fps:.2f}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    cv2.imshow(filename, frame)

    if not drowning_alert:
        stop_siren()
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
stop_siren()
cv2.destroyAllWindows()

# Relabelling for YOLO Training

In [4]:
import os
import shutil

yolo_training_data_dir = os.getenv('YOLO_TRAINING_DATA_DIR')
raw_data_dir = os.getenv('RAW_DATA_DIR')
img_dir = os.getenv('IMG_DIR')
label_dir = os.getenv('LABEL_DIR')
train_dir = os.getenv('TRAIN_DIR')
val_dir = os.getenv('VAL_DIR')
yolo_config_dir = os.getenv('YOLO_CONFIG_DIR')

In [ ]:
for split in [train_dir, val_dir]:
    img_src_dir = os.path.join(raw_data_dir, img_dir, split)
    label_src_dir = os.path.join(raw_data_dir, label_dir, split)

    img_dst_dir = os.path.join(yolo_training_data_dir, img_dir, split)
    label_dst_dir = os.path.join(yolo_training_data_dir, label_dir, split)

    os.makedirs(img_dst_dir, exist_ok=True)
    os.makedirs(label_dst_dir, exist_ok=True)

    for filename in os.listdir(img_src_dir):
        if filename.endswith(('.jpg', )):
            shutil.copy2(os.path.join(img_src_dir, filename), os.path.join(img_dst_dir, filename))

            label_filename = os.path.splitext(filename)[0] + '.txt'
            label_src_path = os.path.join(label_src_dir, label_filename)
            label_dst_path = os.path.join(label_dst_dir, label_filename)

            if os.path.exists(label_src_path):
                with open(label_src_path, 'r') as f:
                    lines = f.readlines()
                new_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        parts[0] = '0'  # change class ID to 0
                        new_lines.append(' '.join(parts))
                with open(label_dst_path, 'w') as f:
                    f.writelines(line + '\n' for line in new_lines)
            else:
                # If no label exists, create an empty label file (optional)
                open(label_dst_path, 'w').close()


# YOLO Fine-Tuning

In [5]:
yaml_content = (
f"""# Dataset paths
path: {yolo_training_data_dir}
train: {os.path.join(img_dir, train_dir)}
val: {os.path.join(img_dir, val_dir)}

# Classes
nc: 1
names: ['person']
""")

yaml_path = os.path.join(yolo_config_dir, "fine_tuning.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)
print(f"Created data configuration at {yaml_path}")

Created data configuration at C:\Users\PC\Downloads\Jie Shen\Drowning-Detection\config\yolo\fine_tuning.yaml


In [7]:
from ultralytics import YOLO
import torch

model = YOLO(os.path.join(os.getenv('YOLO_DIR'), 'yolo11n.pt'))

results = model.train(
    data=yaml_path,
    epochs=50,
    patience=10,
    batch=16,
    imgsz=640,
    save=True,
    save_period=1,
    cache=False,
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=12,
    project=os.getenv('YOLO_DIR'),
    name='yolo11n-finetuned',
    exist_ok=True,
    optimizer='auto',
    seed=32,
    deterministic=False,
    single_cls=False,
    resume=False,
    fraction=1,
    dropout=0,
    val=True,
    plots=True,
    degrees=10,
    mosaic=1
)

New https://pypi.org/project/ultralytics/8.3.109 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.85  Python-3.11.11 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\PC\Downloads\Jie Shen\Drowning-Detection\models\YOLO\yolo11n.pt, data=C:\Users\PC\Downloads\Jie Shen\Drowning-Detection\config\yolo\fine_tuning.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=0, workers=12, project=C:\Users\PC\Downloads\Jie Shen\Drowning-Detection\models\YOLO, name=yolo11n-finetuned, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=32, deterministic=False, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, 

train: Scanning C:\Users\PC\Downloads\Jie Shen\Drowning-Detection\data\yolo_training\labels\train.cache... 7000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7000/7000 [00:00<?, ?it/s]
val: Scanning C:\Users\PC\Downloads\Jie Shen\Drowning-Detection\data\yolo_training\labels\val.cache... 1572 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1572/1572 [00:00<?, ?it/s]


Plotting labels to C:\Users\PC\Downloads\Jie Shen\Drowning-Detection\models\YOLO\yolo11n-finetuned\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 12 dataloader workers
Logging results to C:\Users\PC\Downloads\Jie Shen\Drowning-Detection\models\YOLO\yolo11n-finetuned
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      2.31G      1.431      1.448      1.148         22        640: 100%|██████████| 438/438 [00:37<00:00, 11.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:05<00:00,  9.93it/s]


                   all       1572       2317      0.975      0.977      0.991      0.652

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      2.32G     0.8791     0.3507     0.9463         12        640: 100%|██████████| 438/438 [00:32<00:00, 13.47it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:03<00:00, 12.50it/s]

                   all       1572       2317      0.994      0.999      0.994      0.805



50 epochs completed in 0.550 hours.
Optimizer stripped from C:\Users\PC\Downloads\Jie Shen\Drowning-Detection\models\YOLO\yolo11n-finetuned\weights\last.pt, 5.4MB
Optimizer stripped from C:\Users\PC\Downloads\Jie Shen\Drowning-Detection\models\YOLO\yolo11n-finetuned\weights\best.pt, 5.4MB

Validating C:\Users\PC\Downloads\Jie Shen\Drowning-Detection\models\YOLO\yolo11n-finetuned\weights\best.pt...
Ultralytics 8.3.85  Python-3.11.11 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:04<00:00, 10.72it/s]


                   all       1572       2317      0.993      0.998      0.994      0.806
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to C:\Users\PC\Downloads\Jie Shen\Drowning-Detection\models\YOLO\yolo11n-finetuned
